In [1]:
import numpy as np
import os

In [2]:
def load_data(folder_path):
    x_train = np.load(os.path.join(folder_path, 'x_train.npy'))
    y_train = np.load(os.path.join(folder_path, 'y_train.npy'))
    x_test = np.load(os.path.join(folder_path, 'x_test.npy'))
    y_test = np.load(os.path.join(folder_path, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [3]:
x_train, y_train, x_test, y_test = load_data('lr4_dataset/')

В данной лабораторной работе будет практиковаться поиск гиперпараметров. Буду рассмотрены алгоритмы поиска гиперпараметров: grid search, random search.

Помимо поиска гиперпараметров будет рассмотрен алгоритм кросс-валидации, позволяющий получить более достоверную оценку качества модели в условиях недостатка данных.
Хотя в работе предоставлена тестовая выборка, здесь она имеет сугубо теоретический характер (для получения финальной оценки) и на практике как правило недоступна. Поэтому во время подбора гиперпараметров используются лишь `x_train, y_train`. `x_test, y_test` используются лишь для получения финальной оценки, чтобы можно было видеть разницу между разными алгоритмами подбора гиперпараметров (если она будет).

Выберите одну модель из списка: MLPClassifier, SGDClassifier, DecisionTreeClassifier, RandomForestClassifier, SVC.
Для выбранной модели произведите поиск оптимальных гиперпараметров.

**Требование**: поиск должен идти как минимум для двух гиперпараметров.

**Требование**: в конструктор моделей передавайте `random_state=1` для воспроизводимости результатов.

## 0. Обучение бейзлайн модели для проведения сравнения

In [38]:
# Обучите базовую модель без изменения гиперпараметров (т.е. используются гиперпараметры по умолчанию).
# Проанализируйте качество модели (accuracy, матрица ошибок).

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

baseline_model = RandomForestClassifier(random_state=1)
baseline_model.fit(x_train, y_train)

mlp_model = MLPClassifier(random_state=1)
mlp_model.fit(x_train, y_train)

svc_model = SVC(random_state=1)
svc_model.fit(x_train, y_train)

base_pred_list = baseline_model.predict(x_test)
mpl_pred_list = mlp_model.predict(x_test)
svc_pred_list = svc_model.predict(x_test)

print(f"Точность RandomForestClassifier {accuracy_score(y_test, base_pred_list):.4f}")
print(f"Матрица ошибок RandomForestClassifier\n{confusion_matrix(y_test, base_pred_list)}")

print(f"\nТочность для MLPClassifier {accuracy_score(y_test, mpl_pred_list):.4f}")
print(f"Матрица ошибок для MLPClassifier\n{confusion_matrix(y_test, mpl_pred_list)}")

print(f"\nТочность SVC {accuracy_score(y_test, svc_pred_list):.4f}")
print(f"Матрица ошибок for SVC\n{confusion_matrix(y_test, svc_pred_list)}")

Точность RandomForestClassifier 0.6333
Матрица ошибок RandomForestClassifier
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 1 1 0]
 [0 0 0 0 3 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 1]
 [1 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 1 0 0 0 0 2 0]
 [0 0 0 0 1 0 0 1 0 1]]

Точность для MLPClassifier 0.5667
Матрица ошибок для MLPClassifier
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [1 0 0 2 0 0 0 0 0 0]
 [0 0 0 2 0 1 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [1 0 0 0 0 1 0 0 1 0]
 [1 0 0 0 0 1 0 1 0 0]]

Точность SVC 0.7333
Матрица ошибок for SVC
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 1 0 1]]


## 1. K-Fold Cross-Validation

In [44]:
# Реализуйте фунцию кросс-валидации
from sklearn.model_selection import KFold
# Замечание: x_test, y_test не должны применятся в рамках данной функции.

def kfold_cv(model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5) -> float:
    """
    Parameters
    ----------
    model_fn : callable
        Функция-фабрика, что конструирует и возвращает новый объект модели.
        Например: `lambda: MLPClassifier(hidden_layer_sizes=(256,))`.
    eval_fn : callable
        Функция вида `eval_fn(labels, predictions)`, что возвращает скаляр (значение метрики).
    x : np.ndarray
        Набор признаков (размерность NxD, N - количество экземпляров, D - количество признаков).
    y : np.ndarray
        Набор меток (размерность N)
    n_splits : int, optional
        Количество фолдов (подвыборок), по умолчанию 5.

    Returns
    -------
    float
        Среднее значение метрики (что вычисляется eval_fn) по фолдам.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    scores = []

    for train_index, val_index in kf.split(x): 
        x_train, x_val = x[train_index], x[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model = model_fn()
        model.fit(x_train, y_train)

        y_pred = model.predict(x_val)
        score = eval_fn(y_val, y_pred)
        scores.append(score)

    return np.mean(scores)

In [45]:

model_factory = lambda: RandomForestClassifier(random_state=1)

evaluation_function = lambda y_true, y_pred: accuracy_score(y_true, y_pred)
mean_score = kfold_cv(model_factory, evaluation_function, x_train, y_train, n_splits=5)
print(f"Средняя точность RandomForestClassifier: {mean_score:.4f}")

model_fact_mlp = lambda: MLPClassifier(random_state=1)

evaluation_function_mlp = lambda y_true, y_pred: accuracy_score(y_true, y_pred)
mean_score_mlp = kfold_cv(model_fact_mlp, evaluation_function_mlp, x_train, y_train, n_splits=5)
print(f"Средняя точность MLPClassifier: {mean_score_mlp:.4f}")

model_fact_svc = lambda: SVC(random_state=1)

evaluation_function_svc = lambda y_true, y_pred: accuracy_score(y_true, y_pred)
mean_score_svc = kfold_cv(model_fact_mlp, evaluation_function_svc, x_train, y_train, n_splits=5)
print(f"Средняя точность SVC: {mean_score_svc:.4f}")


Средняя точность RandomForestClassifier: 0.3364
Средняя точность MLPClassifier: 0.4455
Средняя точность SVC: 0.4455


## 2. Grid search

In [46]:
# 1. Реализуйте алгоритм поиска гиперпараметров grid search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.

from itertools import product

def grid_search(param_grid, model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5):

    best_params = None
    best_score = -np.inf
    all_scores = []

    # Перебираем все комбинации параметров
    for params in product(*param_grid.values()):
        current_params = dict(zip(param_grid.keys(), params))
        model_factory = lambda: model_fn(**current_params)

        score = kfold_cv(model_factory, eval_fn, x, y, n_splits)
        all_scores.append((current_params, score))

        if score > best_score:
            best_score = score
            best_params = current_params

    return {"best_params": best_params, "best_score": best_score, "all_scores": all_scores}


In [47]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.

from time import time

model_fn = RandomForestClassifier
eval_fn = accuracy_score

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt", "log2", None]
}

start_time = time()
results = grid_search(param_grid, model_fn, eval_fn, x_train, y_train, n_splits=5)
elapsed_time = time() - start_time

print(f"Лучшие параметры: {results['best_params']}")
print(f"Лучшая точность: {results['best_score']:.4f}")
print(f"Время работы: {elapsed_time:.2f} секунд")

best_model = model_fn(**results['best_params'])
best_model.fit(x_train, y_train)

y_test_pred = best_model.predict(x_test)
print(f"Точность на тестовой выборке: {eval_fn(y_test, y_test_pred):.4f}")

Лучшие параметры: {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None}
Лучшая точность: 0.4727
Время работы: 241.69 секунд
Точность на тестовой выборке: 0.6333


In [48]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.

from time import time

model_fn = MLPClassifier
eval_fn = accuracy_score

mlp_param_grid = {
    "hidden_layer_sizes": [(50,), (100,), (100, 50)],
    "activation": ["tanh", "relu"],
    "solver": ["adam", "sgd"],
    "learning_rate": ["constant", "adaptive"]
}

start_time = time()
results = grid_search(mlp_param_grid, model_fn, eval_fn, x_train, y_train, n_splits=5)
elapsed_time = time() - start_time

print(f"Best Parameters: {results['best_params']}")
print(f"Best Score: {results['best_score']:.4f}")
print(f"Elapsed Time: {elapsed_time:.2f} секунд")

best_model = model_fn(**results['best_params'])
best_model.fit(x_train, y_train)

y_test_pred = best_model.predict(x_test)
print(f"Accuracy of MLPClassifier on test data: {eval_fn(y_test, y_test_pred):.4f}")


c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

Best Parameters: {'hidden_layer_sizes': (100,), 'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'constant'}
Best Score: 0.5545
Elapsed Time: 23.25 секунд
Accuracy of MLPClassifier on test data: 0.7333


c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [49]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.

from time import time

model_fn = SVC
eval_fn = accuracy_score

svc_param_grid = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf", "poly"],
    "gamma": ["scale", "auto"]
}

start_time = time()
results = grid_search(svc_param_grid, model_fn, eval_fn, x_train, y_train, n_splits=5)
elapsed_time = time() - start_time

print(f"Лучшие параметры: {results['best_params']}")
print(f"Лучшая точность: {results['best_score']:.4f}")
print(f"Вребя работы: {elapsed_time:.2f} секунд")

best_model_svc = model_fn(**results['best_params'])
best_model_svc.fit(x_train, y_train)

y_test_pred = best_model.predict(x_test)
print(f"Точность SVC модели на тестовой выборке: {eval_fn(y_test, y_test_pred):.4f}")


Лучшие параметры: {'C': 10, 'kernel': 'rbf', 'gamma': 'scale'}
Лучшая точность: 0.5909
Вребя работы: 0.29 секунд
Точность SVC модели на тестовой выборке: 0.7333


#### В пункте 0 самый лучший результат показала модель SVC (accur=0.73), на втором месте идёт RandomForestClassifer (accur=0.63) и самой неудачной оказалась MLPClassifier (accur=0.57)

#### При использовании grid search особых изменений не произошло, только MLPClassifier улучшила точность (accur=0.73)

## 3. Random search

In [50]:
# 1. Реализуйте алгоритм поиска гиперпараметров random search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.
# Требование: количество итераций должно быть меньше в сравнении с grid search.

import random

def random_search(param_dist, model_fn, eval_fn, x, y, n_iter=10, n_splits=5):
    best_score = -np.inf
    best_params = None

    start_time = time()

    for _ in range(n_iter):
        # Генерация случайной комбинации параметров
        random_params = {key: random.choice(values) for key, values in param_dist.items()}

        # Оценка качества модели через kfold_cv
        score = kfold_cv(lambda: model_fn(random_params), eval_fn, x, y, n_splits=n_splits)

        if score > best_score:
            best_score = score
            best_params = random_params

    elapsed_time = time() - start_time

    print(f"Лучшие параметры: {best_params}")
    print(f"Лучший средняя точность: {best_score:.4f}")
    print(f"Время поиска параметров: {elapsed_time:.2f} секунд")

    return best_params, best_score


param_dist = {
        "n_estimators": [50, 100, 200, 300],
        "max_depth": [10, 20, 30, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "bootstrap": [True, False],
    }

model_factory = lambda params: RandomForestClassifier(random_state=1, **params)
evaluation_function = lambda y_true, y_pred: accuracy_score(y_true, y_pred)

best_params, _ = random_search(param_dist, model_factory, evaluation_function, x_train, y_train, n_iter=20, n_splits=5)


Лучшие параметры: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'bootstrap': True}
Лучший средняя точность: 0.4091
Время поиска параметров: 34.53 секунд


In [51]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

best_model_rfc = RandomForestClassifier(random_state=1, **best_params)
best_model_rfc.fit(x_train, y_train)

y_test_pred = best_model_rfc.predict(x_test)

print(f"Test accuracy: {accuracy_score(y_test, y_test_pred):.4f}")


Test accuracy: 0.6667


In [52]:
mlp_param_grid = {
    "hidden_layer_sizes": [(50,), (100,), (100, 50)],
    "activation": ["tanh", "relu"],
    "solver": ["adam", "sgd"],
    "learning_rate": ["constant", "adaptive"]
}

model_factory_mlp = lambda params: MLPClassifier(random_state=1, **params)
evaluation_function_mlp = lambda y_true, y_pred: accuracy_score(y_true, y_pred)

best_params_mlp, _ = random_search(mlp_param_grid, model_factory_mlp,\
                                   evaluation_function_mlp, x_train, y_train,\
                                   n_iter=20, n_splits=5)


c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

Лучшие параметры: {'hidden_layer_sizes': (50,), 'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'adaptive'}
Лучший средняя точность: 0.5364
Время поиска параметров: 18.61 секунд


c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [53]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

best_model_mlp = MLPClassifier(random_state=1, **best_params_mlp)
best_model_mlp.fit(x_train, y_train)

y_test_pred = best_model_mlp.predict(x_test)

print(f"Test accuracy of MLPClassifier: {accuracy_score(y_test, y_test_pred):.4f}")


Test accuracy of MLPClassifier: 0.5667


c:\Users\gresk\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [54]:
svc_param_grid = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf", "poly"],
    "gamma": ["scale", "auto"]
}

model_factory_svc = lambda params: SVC(random_state=1, **params)
evaluation_function_svc = lambda y_true, y_pred: accuracy_score(y_true, y_pred)

best_params_svc, _ = random_search(svc_param_grid, model_factory_svc,\
                                   evaluation_function_svc, x_train, y_train,\
                                   n_iter=20, n_splits=5)


Лучшие параметры: {'C': 10, 'kernel': 'rbf', 'gamma': 'scale'}
Лучший средняя точность: 0.5909
Время поиска параметров: 0.32 секунд


In [55]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

best_model_svc = SVC(random_state=1, **best_params_svc)
best_model_svc.fit(x_train, y_train)

y_test_pred = best_model_svc.predict(x_test)

print(f"Тестовая точность SVC модели: {accuracy_score(y_test, y_test_pred):.4f}")


Тестовая точность SVC модели: 0.8000


#### Random search не помог повысить точность в RandomForestClassifer и MLPClassifier (в этом случае точность понизилась), но зато слегка увеличил точность SVC model (accur=0.77)

#### Но при этом с Random search уменьшилось время затрачиваемое на поиск подходящих гиперпараметров

## 4. Доп. задание (опционально)

### 4.1 Bayesian optimization

Примените байесовскую оптимизацию для поиска гиперпараметров.
В качестве алгоритма используйте `BayesSearchCV` из пакета `scikit-optimize`.

Сложность: почти бесплатный балл.

In [56]:
# 1. Инстанцируйте BayesSearchCV.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.

from skopt import BayesSearchCV

svc_param_space = {
    "C": (0.1, 100, "log-uniform"),  # Коэффициент регуляризации (логарифмический диапазон)
    "gamma": (0.001, 1, "log-uniform"),  # Коэффициент ядра (логарифмический диапазон)
    "kernel": ["linear", "rbf", "poly"],  # Тип ядра
    "degree": (2, 5)  # Степень для полиномиального ядра (если используется)
}


model = SVC(random_state=1)

bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=svc_param_space,
    n_iter=25,
    cv=5,
    random_state=1,
    scoring='accuracy',
    n_jobs=-1  # Используем все доступные потоки
)

start_time = time()
bayes_search.fit(x_train, y_train)
elapsed_time = time() - start_time

print(f"Лучшие гиперпараметры: {bayes_search.best_params_}")
print(f"Лучшая CV точность: {bayes_search.best_score_:.4f}")
print(f"Время работы: {elapsed_time:.2f} секунд")


Лучшие гиперпараметры: OrderedDict([('C', 11.237567661049233), ('degree', 3), ('gamma', 0.0012252759808604556), ('kernel', 'rbf')])
Лучшая CV точность: 0.6364
Время работы: 21.83 секунд


In [57]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

best_model = bayes_search.best_estimator_
best_model.fit(x_train, y_train)

y_test_pred = best_model.predict(x_test)

print(f"Точность с bayes_search на тестовом наборе: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Матрица ошибок:\n{confusion_matrix(y_test, y_test_pred)}")

Точность с bayes_search на тестовом наборе: 0.8000
Матрица ошибок:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]


#### С bayes_search была улучшена точность SVC model (accur=0.8), но при этом поиск подходящих гиперпараметров занял 28 секунд. Показатель точности с bayes_search значительно высок, чем этот же показатель без поиска гиперпараметров

### 4.2 Tree of Parzen Estimators (TPE) из HyperOpt

Примените TPE из библиотеки hyperopt для поиска гиперпараметров. Вики по HyperOpt: https://github.com/hyperopt/hyperopt/wiki/FMin

Сложность: чтец документаций o(*￣▽￣*)ブ.

In [58]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

def objective(args):
    # Принимает гиперпараметры, инстанцирует модель, обучает её, возвращает значение метрики.
    # Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.

    model = SVC(
        C=args['C'],
        kernel=args['kernel'],
        gamma=args['gamma'],
        random_state=1
    )

    accuracy = kfold_cv( # Используем kfold_cv для оценки метрики
        model_fn=lambda: model,
        eval_fn=lambda y_true, y_pred: accuracy_score(y_true, y_pred),
        x=x_train,
        y=y_train,
        n_splits=5
    )

    # Функция должна минимизировать loss, поэтому используем 1 - accuracy
    return {'loss': 1 - accuracy, 'status': STATUS_OK}

In [59]:
# Определите пространство поиска гиперпараметров
space = {
    'C': hp.loguniform('C', np.log(0.01), np.log(100)),
    'kernel': hp.choice('kernel', ['linear', 'rbf']),
    'gamma': hp.loguniform('gamma', np.log(0.001), np.log(1))
}

In [60]:
# 1. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 2. Выведите найденные значения гиперпараметров и время работы.

trials = Trials()

start_time = time()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=30,
    trials=trials
)
elapsed_time = time() - start_time

# Преобразование результатов best_params
final_params = {
    'C': best_params['C'],
    'kernel': ['linear', 'rbf'][best_params['kernel']],
    'gamma': best_params['gamma']
}

print("Оптимальные гиперпараметры:", final_params)
print(f"Время поиска {elapsed_time} секунд")


100%|██████████| 30/30 [00:00<00:00, 50.16trial/s, best loss: 0.4181818181818182]
Оптимальные гиперпараметры: {'C': np.float64(0.024973156911813166), 'kernel': 'linear', 'gamma': np.float64(0.044955656954604996)}
Время поиска 0.6031386852264404 секунд


In [61]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

final_model = SVC(
    C=final_params['C'],
    kernel=final_params['kernel'],
    gamma=final_params['gamma'],
    random_state=1
)
final_model.fit(x_train, y_train)

# Оцениваем качество на тестовой выборке
y_pred = final_model.predict(x_test)

print(f"Точность hyperopt: {accuracy_score(y_test, y_pred):.4f}")
print(f"Матрица ошибок:\n{confusion_matrix(y_test, y_pred)}")

Точность hyperopt: 0.7667
Матрица ошибок:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 0 2 0 0 1 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 1 0 1 0 1]]


#### Использование библиотеки hyperopt позволяет подобрать оптимальные для модели гиперпараметры за очень короткий промежуток времени и показывает хорошую точность (accur=0.8). Результаты с использованием данной библиотеки оказались самыми лучшими по сравнению со всеми прошлыми пунктами (0, 2, 3)